In [ ]:
import csv
import os
import warnings
import pickle
import pandas as pd
import numpy as np

In a flattened array, we have a lower triangular part WITHOUT diagonal. Therefore, we need to fill in the upper triangular part and add diagonal values which will be = 1
This is because niliearn vectorized matrices itself as follows: https://nilearn.github.io/stable/modules/generated/nilearn.connectome.ConnectivityMeasure.html

# Get 2D matrix from 1D array and add labels: Glasser + MSA-IV

# Here I do it for 5 subjects, but you can do for all (just dont have enough RAM)

In [ ]:
# Upload csv
glasser_Tian_s4_fcorr = pd.read_csv('/ML_DATASETS/Brain/rsMRI/bulk_timeseries/matrices/glasser/glasser_Tian_s4_arrays_full_correlation.csv', nrows = 3)
glasser_Tian_s4_fcorr_id = glasser_Tian_s4_fcorr['Unnamed: 0']
glasser_Tian_s4_fcorr = glasser_Tian_s4_fcorr.drop(columns='Unnamed: 0')

In [ ]:
# Transform 1D array into 2D matrix
vectorized_array = glasser_Tian_s4_fcorr.to_numpy()[0]
matrix_size = int(np.ceil((1 + np.sqrt(1 + 8 * len(vectorized_array))) / 2))
glasser_Tian_s4_matrix = np.zeros((matrix_size, matrix_size))
index = 0
for i in range(1, matrix_size):
    for j in range(i):
        glasser_Tian_s4_matrix[i, j] = vectorized_array[index]
        index += 1

# Mirror the lower triangular part to the upper triangular part
glasser_Tian_s4_matrix += glasser_Tian_s4_matrix.T

# Add a diagonal of 1s
np.fill_diagonal(glasser_Tian_s4_matrix, 1)
glasser_Tian_s4_matrix

### Get label names from a timeseries file

In [ ]:
# Timeseries file, if needed
with open('/ML_DATASETS/Brain/rsMRI/bulk_timeseries/merged_dict/glasser/glasser_Tian_s4_dict_df.pkl', "rb") as a:
    glasser_Tian_s4_timeseries = pickle.load(a)
labels = list(glasser_Tian_s4_timeseries.values())[0]['label_name'].to_list()

### Add labels to the matrix

In [ ]:
glasser_Tian_s4_fcorr_matrix_labels = pd.DataFrame(glasser_Tian_s4_matrix, columns = labels, index = labels)
glasser_Tian_s4_fcorr_matrix_labels

##############################################################################################

# An example for Schaefer17n100p + MSA-I which does not have labels inside a main dictionary

In [ ]:
# Upload csv
Schaefer17n100p_tian_s1_fcorr = pd.read_csv('/ML_DATASETS/Brain/rsMRI/bulk_timeseries/matrices/Schaefer/17n/Schaefer17n100p_tian_s1_arrays_full_correlation.csv')
Schaefer17n100p_tian_s1_fcorr_id = Schaefer17n100p_tian_s1_fcorr['Unnamed: 0']
Schaefer17n100p_tian_s1_fcorr = Schaefer17n100p_tian_s1_fcorr.drop(columns='Unnamed: 0')

In [ ]:
Schaefer17n100p_tian_s1_fcorr

Transform 1D array into 2D matrix

In [ ]:
# Transform 1D array into 2D matrix
vectorized_array = Schaefer17n100p_tian_s1_fcorr.to_numpy()[0]
matrix_size = int(np.ceil((1 + np.sqrt(1 + 8 * len(vectorized_array))) / 2))
# alternatively: n = int((1 + np.sqrt(1 + 8 * len(lower_triangular))) / 2)
Schaefer17n100p_tian_s1_fcorr_matrix = np.zeros((matrix_size, matrix_size))
index = 0
for i in range(1, matrix_size):
    for j in range(i):
        Schaefer17n100p_tian_s1_fcorr_matrix[i, j] = vectorized_array[index]
        index += 1

# Mirror the lower triangular part to the upper triangular part
Schaefer17n100p_tian_s1_fcorr_matrix += Schaefer17n100p_tian_s1_fcorr_matrix.T

# Add a diagonal of 1s
np.fill_diagonal(Schaefer17n100p_tian_s1_fcorr_matrix, 1)
Schaefer17n100p_tian_s1_fcorr_matrix

In [ ]:
# Alternative way
# Fill the lower triangular part of the matrix
tril_indices = np.tril_indices(n, -1)
Schaefer17n100p_tian_s1_fcorr_matrix[tril_indices] = vectorized_array

# Since the matrix is symmetric, copy the lower triangular part to the upper triangular part
matrix = Schaefer17n100p_tian_s1_fcorr_matrix + Schaefer17n100p_tian_s1_fcorr_matrix.T

### Get label names from a timeseries file

In [ ]:
# Upload timeseries file: cortical
with open('/ML_DATASETS/Brain/rsMRI/bulk_timeseries/Schaefer/timeseries_Schaefer17n100p_full_dict.pkl', "rb") as a:
    timeseries_Schaefer17n100p_full_dict = pickle.load(a)
timeseries_Schaefer17n100p_full_dict

In [ ]:
cortical_labels = list(timeseries_Schaefer17n100p_full_dict.values())[0]['label_name']
cortical_labels

In [ ]:
# Upload timeseries file: subcortical
timeseries_Tian_s1_full_dict = np.load('/ML_DATASETS/Brain/rsMRI/bulk_timeseries/timeseries_tian_s1_full_dict.npy', allow_pickle=True).item()
subcortical_labels = list(timeseries_Tian_s1_full_dict.values())[0]['label_name']
subcortical_labels

Convert labels to list and combine them nicely

In [ ]:
Schaefer17n100p_tian_s1_labels = subcortical_labels.to_list()
Schaefer17n100p_tian_s1_labels.append(cortical_labels.to_list())

In [ ]:
Schaefer17n100p_tian_s1_labels_list = []
for item in Schaefer17n100p_tian_s1_labels:
    if isinstance(item, list):
        Schaefer17n100p_tian_s1_labels_list.extend(item)
    else:
        Schaefer17n100p_tian_s1_labels_list.append(item)

### Add labels to the matrix

In [ ]:
Schaefer17n100p_tian_s1_fcorr_matrix_labels = pd.DataFrame(Schaefer17n100p_tian_s1_fcorr_matrix, columns = Schaefer17n100p_tian_s1_labels_list, index = Schaefer17n100p_tian_s1_labels_list)
Schaefer17n100p_tian_s1_fcorr_matrix_labels

In [ ]:
Schaefer17n100p_tian_s1_fcorr_matrix_labels

#### *You can also extract label names for a 1D array and add them to the array as column names

Get label names

In [ ]:
# Get combinations of label names: way 1
data_frame = list(glasser_Tian_s4_timeseries.values())[0]
label_names = data_frame.iloc[:, 0].tolist()

pairwise_combinations = []

# Generate the pairwise combinations in the specified order
for i in range(1, len(label_names)):
    for j in range(i):
        pairwise_combinations.append(f"{label_names[i]} & {label_names[j]}")
for combination in pairwise_combinations:
    print(combination)
pairwise_combinations_df = pd.DataFrame(pairwise_combinations)
pairwise_combinations_df[0]

In [ ]:
# Get combinations of label names: way 2
from itertools import combinations
pairwise_combinations = list(combinations(label_names, 2))

# Print the pairwise combinations
for pair in pairwise_combinations:
    print(f"{pair[0]} & {pair[1]}")